# Flood Impact Assessment in Emilia Romagna using Sentinel-1 SAR Data

This notebook demonstrates flood impact assessment in Emilia Romagna, Italy using Sentinel-1 SAR data accessed through the openEO API. The analysis focuses on the severe flooding events that occurred around May 16-17, 2023.

## Background
In May 2023, Emilia Romagna experienced severe flooding due to intense rainfall, affecting thousands of people and causing significant damage to infrastructure and agriculture. SAR data is particularly valuable for flood monitoring because:

- **Weather Independence**: SAR can penetrate clouds and operate day/night
- **Water Detection**: Water appears dark in SAR images due to specular reflection
- **Change Detection**: Comparing pre- and post-flood images reveals flooded areas

## Requirements
- SAR4CET toolkit (will be installed below)
- openEO Python client
- Copernicus Dataspace account for authentication
- xarray, matplotlib, numpy for data processing and visualization

## 0. Setup and Installation

First, let's clone the SAR4CET repository and install all required dependencies.

In [ ]:
# Clone the SAR4CET repository
import os
import subprocess
import sys

# Check if SAR4CET directory already exists
if not os.path.exists('SAR4CET'):
    print('Cloning SAR4CET repository...')
    result = subprocess.run(['git', 'clone', 'https://github.com/naikp13/SAR4CET.git'], 
                          capture_output=True, text=True)
    if result.returncode == 0:
        print('Repository cloned successfully!')
    else:
        print(f'Error cloning repository: {result.stderr}')
else:
    print('SAR4CET directory already exists.')

In [ ]:
# Install requirements from requirements.txt
print('Installing SAR4CET requirements...')

# Change to SAR4CET directory and install requirements
if os.path.exists('SAR4CET/requirements.txt'):
    result = subprocess.run([sys.executable, '-m', 'pip', 'install', '-r', 'SAR4CET/requirements.txt'], 
                          capture_output=True, text=True)
    if result.returncode == 0:
        print('Requirements installed successfully!')
        print('Installed packages from requirements.txt')
    else:
        print(f'Error installing requirements: {result.stderr}')
        print('Trying to install individual packages...')
        # Try installing key packages individually
        key_packages = ['openeo>=0.22.0', 'xarray>=0.19.0', 'numpy>=1.20.0', 
                       'matplotlib>=3.4.0', 'rasterio>=1.2.0', 'scipy>=1.7.0']
        for package in key_packages:
            subprocess.run([sys.executable, '-m', 'pip', 'install', package])
else:
    print('requirements.txt not found, installing key packages manually...')
    key_packages = ['openeo>=0.22.0', 'xarray>=0.19.0', 'numpy>=1.20.0', 
                   'matplotlib>=3.4.0', 'rasterio>=1.2.0', 'scipy>=1.7.0']
    for package in key_packages:
        subprocess.run([sys.executable, '-m', 'pip', 'install', package])

In [ ]:
# Add SAR4CET to Python path for imports
import sys
import os

# Add SAR4CET directory to Python path
sar4cet_path = os.path.abspath('SAR4CET')
if sar4cet_path not in sys.path:
    sys.path.insert(0, sar4cet_path)
    print(f'Added {sar4cet_path} to Python path')

# Verify installation by importing key modules
try:
    import openeo
    import xarray as xr
    import matplotlib.pyplot as plt
    import numpy as np
    from scipy import ndimage
    print('All required packages imported successfully!')
    print(f'openEO version: {openeo.__version__}')
    print(f'xarray version: {xr.__version__}')
except ImportError as e:
    print(f'Import error: {e}')
    print('Please install missing packages manually.')

## Import Required Libraries

Now let's import all the libraries we'll need for the flood impact analysis.

In [ ]:
import openeo
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime, timedelta
import pandas as pd
from scipy import ndimage
from scipy.stats import zscore
import warnings
warnings.filterwarnings('ignore')

# Set matplotlib parameters for better plots
plt.rcParams['figure.figsize'] = (14, 10)
plt.rcParams['font.size'] = 11

# Optional: Import SAR4CET modules if available
try:
    from sar4cet import preprocessing, visualization
    print('SAR4CET modules imported successfully!')
except ImportError:
    print('SAR4CET modules not available, using standalone approach.')

## 1. Connect to openEO Backend

Connect to the Copernicus Dataspace openEO backend and authenticate.

In [ ]:
# Connect to Copernicus Dataspace openEO backend
backend = 'openeo.dataspace.copernicus.eu'
conn = openeo.connect(backend).authenticate_oidc()
print(f'Connected to {backend}')

## 2. Define Area of Interest and Time Periods

We'll focus on the area around Ravenna and Faenza in Emilia Romagna, which were severely affected by the May 2023 floods.

In [ ]:
# Area of Interest: Emilia Romagna flood-affected region
# Centered around Ravenna-Faenza area (44.3640607, 12.0590095)
spatial_extent = {
    'west': 11.8000,   # Western boundary
    'east': 12.3000,   # Eastern boundary
    'south': 44.1000,  # Southern boundary
    'north': 44.6000,  # Northern boundary
    'crs': 'EPSG:4326',
}

# Define time periods for flood analysis
# Pre-flood period (stable conditions)
pre_flood_period = ['2023-05-01', '2023-05-15']

# Post-flood period (during and after flooding)
post_flood_period = ['2023-05-17', '2023-05-31']

# Reference period for comparison (same time previous year)
reference_period = ['2022-05-01', '2022-05-31']

print(f'Area of Interest: {spatial_extent}')
print(f'Pre-flood period: {pre_flood_period[0]} to {pre_flood_period[1]}')
print(f'Post-flood period: {post_flood_period[0]} to {post_flood_period[1]}')
print(f'Reference period: {reference_period[0]} to {reference_period[1]}')

# Calculate area coverage
lat_range = spatial_extent['north'] - spatial_extent['south']
lon_range = spatial_extent['east'] - spatial_extent['west']
print(f'\nArea coverage: {lat_range:.2f}° × {lon_range:.2f}° (~{lat_range*111:.0f} × {lon_range*85:.0f} km)')

## 3. Load and Process Sentinel-1 Data

Load Sentinel-1 data for pre-flood, post-flood, and reference periods.

In [ ]:
# Function to load and process SAR data for a given time period
def load_and_process_sar_flood(conn, spatial_extent, temporal_extent, period_name):
    '''Load and process SAR data for flood analysis'''
    print(f'Processing {period_name}: {temporal_extent[0]} to {temporal_extent[1]}')
    
    # Load Sentinel-1 data
    s1 = conn.load_collection(
        'SENTINEL1_GRD',
        spatial_extent=spatial_extent,
        bands=['VV', 'VH'],
        temporal_extent=temporal_extent,
        properties={'sat:orbit_state': lambda od: od == 'ASCENDING'},
    )
    
    # Apply SAR backscatter processing
    s1_scatter = s1.sar_backscatter(
        coefficient='sigma0-ellipsoid', 
        elevation_model='COPERNICUS_30'
    )
    
    # Convert to dB scale
    s1bs = s1_scatter.apply(lambda x: 10 * x.log(base=10))
    
    # Create median composite to reduce speckle
    s1_median = s1bs.median_time()
    
    return s1_median

print('Function defined for SAR flood analysis processing')

In [ ]:
# Process all time periods
print('Starting flood impact data processing...')
print('Note: This process may take 10-15 minutes depending on data availability')

processed_data = {}

# Process pre-flood data
try:
    print('\n=== Processing Pre-flood Data ===')
    pre_flood_data = load_and_process_sar_flood(conn, spatial_extent, pre_flood_period, 'Pre-flood')
    processed_data['pre_flood'] = pre_flood_data
    print('✓ Pre-flood data processed successfully')
except Exception as e:
    print(f'✗ Error processing pre-flood data: {str(e)}')
    processed_data['pre_flood'] = None

# Process post-flood data
try:
    print('\n=== Processing Post-flood Data ===')
    post_flood_data = load_and_process_sar_flood(conn, spatial_extent, post_flood_period, 'Post-flood')
    processed_data['post_flood'] = post_flood_data
    print('✓ Post-flood data processed successfully')
except Exception as e:
    print(f'✗ Error processing post-flood data: {str(e)}')
    processed_data['post_flood'] = None

# Process reference data
try:
    print('\n=== Processing Reference Data ===')
    reference_data = load_and_process_sar_flood(conn, spatial_extent, reference_period, 'Reference')
    processed_data['reference'] = reference_data
    print('✓ Reference data processed successfully')
except Exception as e:
    print(f'✗ Error processing reference data: {str(e)}')
    processed_data['reference'] = None

# Summary
valid_datasets = sum(1 for data in processed_data.values() if data is not None)
print(f'\nProcessing complete: {valid_datasets}/3 datasets successfully processed')

## 4. Download Processed Data

Download the processed SAR data for all time periods.

In [ ]:
# Download all processed datasets
downloaded_files = {}

print('Downloading processed flood analysis data...')

for period_name, data in processed_data.items():
    if data is not None:
        filename = f'emilia_romagna_{period_name}_flood.nc'
        try:
            print(f'Downloading {filename}...')
            data.download(filename)
            downloaded_files[period_name] = filename
            print(f'✓ Downloaded {filename}')
        except Exception as e:
            print(f'✗ Error downloading {filename}: {str(e)}')
            downloaded_files[period_name] = None
    else:
        print(f'Skipping {period_name} - no data available')
        downloaded_files[period_name] = None

valid_downloads = sum(1 for file in downloaded_files.values() if file is not None)
print(f'\nDownload complete: {valid_downloads} files successfully downloaded')

## 5. Load Downloaded Data and Prepare for Analysis

Load all downloaded NetCDF files and prepare them for flood impact analysis.

In [ ]:
# Load all downloaded files
datasets = {}

for period_name, filename in downloaded_files.items():
    if filename is not None:
        try:
            ds = xr.open_dataset(filename)
            datasets[period_name] = ds
            print(f'Loaded {filename} - Shape: {ds.dims}')
        except Exception as e:
            print(f'Error loading {filename}: {str(e)}')
            datasets[period_name] = None
    else:
        datasets[period_name] = None

valid_datasets = sum(1 for ds in datasets.values() if ds is not None)
print(f'\nLoaded {valid_datasets} datasets for flood analysis')

if valid_datasets > 0:
    # Get a sample dataset to check structure
    sample_ds = next(ds for ds in datasets.values() if ds is not None)
    print(f'Dataset dimensions: {sample_ds.dims}')
    print(f'Available bands: {list(sample_ds.data_vars)}')
    print(f'Spatial resolution: ~{abs(sample_ds.x.values[1] - sample_ds.x.values[0]):.6f}° per pixel')
else:
    print('No datasets available for analysis')

## 6. Flood Detection and Change Analysis

Perform flood detection by comparing pre-flood and post-flood SAR backscatter values.

In [ ]:
# Function to detect flood areas using SAR backscatter change
def detect_flood_areas(pre_flood_ds, post_flood_ds, vv_threshold=-3.0, vh_threshold=-2.0):
    '''
    Detect flood areas by comparing pre- and post-flood SAR data
    
    Parameters:
    - pre_flood_ds: Pre-flood dataset
    - post_flood_ds: Post-flood dataset
    - vv_threshold: VV backscatter decrease threshold (dB) for flood detection
    - vh_threshold: VH backscatter decrease threshold (dB) for flood detection
    
    Returns:
    - flood_mask: Boolean array indicating flood areas
    - change_vv: VV backscatter change (post - pre)
    - change_vh: VH backscatter change (post - pre)
    '''
    
    # Calculate backscatter changes
    change_vv = post_flood_ds.VV.values - pre_flood_ds.VV.values
    change_vh = post_flood_ds.VH.values - pre_flood_ds.VH.values
    
    # Detect flood areas (significant decrease in backscatter)
    # Water appears dark in SAR due to specular reflection
    flood_mask_vv = change_vv < vv_threshold
    flood_mask_vh = change_vh < vh_threshold
    
    # Combined flood mask (either VV or VH indicates flooding)
    flood_mask = flood_mask_vv | flood_mask_vh
    
    return flood_mask, change_vv, change_vh, flood_mask_vv, flood_mask_vh

# Perform flood detection if we have both pre- and post-flood data
if datasets['pre_flood'] is not None and datasets['post_flood'] is not None:
    print('Performing flood detection analysis...')
    
    # Detect flood areas
    flood_mask, change_vv, change_vh, flood_mask_vv, flood_mask_vh = detect_flood_areas(
        datasets['pre_flood'], 
        datasets['post_flood'],
        vv_threshold=-3.0,  # 3 dB decrease threshold for VV
        vh_threshold=-2.0   # 2 dB decrease threshold for VH
    )
    
    # Calculate flood statistics
    total_pixels = flood_mask.size
    flood_pixels = np.sum(flood_mask)
    flood_percentage = (flood_pixels / total_pixels) * 100
    
    print(f'Flood detection completed:')
    print(f'  Total pixels: {total_pixels:,}')
    print(f'  Flood-affected pixels: {flood_pixels:,}')
    print(f'  Flood coverage: {flood_percentage:.2f}% of analyzed area')
    
    # Estimate flood area (rough calculation)
    pixel_size_deg = abs(datasets['pre_flood'].x.values[1] - datasets['pre_flood'].x.values[0])
    pixel_area_km2 = (pixel_size_deg * 111) * (pixel_size_deg * 85)  # Rough conversion to km²
    flood_area_km2 = flood_pixels * pixel_area_km2
    
    print(f'  Estimated flood area: {flood_area_km2:.1f} km²')
    
    # Change statistics
    print(f'\nBackscatter change statistics:')
    print(f'  VV change range: {np.nanmin(change_vv):.2f} to {np.nanmax(change_vv):.2f} dB')
    print(f'  VH change range: {np.nanmin(change_vh):.2f} to {np.nanmax(change_vh):.2f} dB')
    print(f'  Mean VV change: {np.nanmean(change_vv):.2f} dB')
    print(f'  Mean VH change: {np.nanmean(change_vh):.2f} dB')
    
else:
    print('Cannot perform flood detection - missing pre-flood or post-flood data')
    flood_mask = None

## 7. Visualization of Flood Impact

Create comprehensive visualizations showing the flood impact and SAR backscatter changes.

In [ ]:
# Visualize pre-flood and post-flood SAR images
if datasets['pre_flood'] is not None and datasets['post_flood'] is not None:
    
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    
    # Pre-flood VV
    im1 = axes[0, 0].imshow(datasets['pre_flood'].VV.values, cmap='gray', vmin=-25, vmax=-5)
    axes[0, 0].set_title('Pre-flood VV Backscatter\n(May 1-15, 2023)')
    axes[0, 0].axis('off')
    cbar1 = plt.colorbar(im1, ax=axes[0, 0], label='Backscatter (dB)')
    
    # Post-flood VV
    im2 = axes[0, 1].imshow(datasets['post_flood'].VV.values, cmap='gray', vmin=-25, vmax=-5)
    axes[0, 1].set_title('Post-flood VV Backscatter\n(May 17-31, 2023)')
    axes[0, 1].axis('off')
    cbar2 = plt.colorbar(im2, ax=axes[0, 1], label='Backscatter (dB)')
    
    # VV Change
    im3 = axes[0, 2].imshow(change_vv, cmap='RdBu_r', vmin=-8, vmax=8)
    axes[0, 2].set_title('VV Backscatter Change\n(Post - Pre)')
    axes[0, 2].axis('off')
    cbar3 = plt.colorbar(im3, ax=axes[0, 2], label='Change (dB)')
    
    # Pre-flood VH
    im4 = axes[1, 0].imshow(datasets['pre_flood'].VH.values, cmap='gray', vmin=-30, vmax=-10)
    axes[1, 0].set_title('Pre-flood VH Backscatter\n(May 1-15, 2023)')
    axes[1, 0].axis('off')
    cbar4 = plt.colorbar(im4, ax=axes[1, 0], label='Backscatter (dB)')
    
    # Post-flood VH
    im5 = axes[1, 1].imshow(datasets['post_flood'].VH.values, cmap='gray', vmin=-30, vmax=-10)
    axes[1, 1].set_title('Post-flood VH Backscatter\n(May 17-31, 2023)')
    axes[1, 1].axis('off')
    cbar5 = plt.colorbar(im5, ax=axes[1, 1], label='Backscatter (dB)')
    
    # VH Change
    im6 = axes[1, 2].imshow(change_vh, cmap='RdBu_r', vmin=-8, vmax=8)
    axes[1, 2].set_title('VH Backscatter Change\n(Post - Pre)')
    axes[1, 2].axis('off')
    cbar6 = plt.colorbar(im6, ax=axes[1, 2], label='Change (dB)')
    
    plt.suptitle('Emilia Romagna Flood Analysis - SAR Backscatter Comparison', fontsize=16, y=0.95)
    plt.tight_layout()
    plt.show()
    
    print('\nInterpretation:')
    print('- Dark areas in SAR images indicate low backscatter (water, smooth surfaces)')
    print('- Bright areas indicate high backscatter (buildings, rough surfaces)')
    print('- Blue areas in change maps show backscatter decrease (potential flooding)')
    print('- Red areas in change maps show backscatter increase')
    
else:
    print('Cannot create SAR comparison plots - missing data')

In [ ]:
# Visualize flood detection results
if flood_mask is not None:
    
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # Combined flood mask
    im1 = axes[0, 0].imshow(flood_mask, cmap='Blues', alpha=0.8)
    axes[0, 0].set_title('Detected Flood Areas\n(Combined VV + VH)')
    axes[0, 0].axis('off')
    
    # VV flood mask
    im2 = axes[0, 1].imshow(flood_mask_vv, cmap='Reds', alpha=0.8)
    axes[0, 1].set_title('VV-based Flood Detection\n(Threshold: -3 dB)')
    axes[0, 1].axis('off')
    
    # VH flood mask
    im3 = axes[1, 0].imshow(flood_mask_vh, cmap='Greens', alpha=0.8)
    axes[1, 0].set_title('VH-based Flood Detection\n(Threshold: -2 dB)')
    axes[1, 0].axis('off')
    
    # Flood overlay on post-flood VV
    axes[1, 1].imshow(datasets['post_flood'].VV.values, cmap='gray', vmin=-25, vmax=-5)
    axes[1, 1].imshow(flood_mask, cmap='Blues', alpha=0.6)
    axes[1, 1].set_title('Flood Areas Overlay\n(Blue = Detected Floods)')
    axes[1, 1].axis('off')
    
    plt.suptitle('Flood Detection Results - May 2023 Emilia Romagna Floods', fontsize=16, y=0.95)
    plt.tight_layout()
    plt.show()
    
    # Create a summary statistics plot
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    # Histogram of VV changes
    axes[0].hist(change_vv.flatten(), bins=50, alpha=0.7, color='blue', edgecolor='black')
    axes[0].axvline(-3, color='red', linestyle='--', linewidth=2, label='Flood threshold (-3 dB)')
    axes[0].set_xlabel('VV Backscatter Change (dB)')
    axes[0].set_ylabel('Number of Pixels')
    axes[0].set_title('Distribution of VV Backscatter Changes')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # Histogram of VH changes
    axes[1].hist(change_vh.flatten(), bins=50, alpha=0.7, color='green', edgecolor='black')
    axes[1].axvline(-2, color='red', linestyle='--', linewidth=2, label='Flood threshold (-2 dB)')
    axes[1].set_xlabel('VH Backscatter Change (dB)')
    axes[1].set_ylabel('Number of Pixels')
    axes[1].set_title('Distribution of VH Backscatter Changes')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
else:
    print('Cannot create flood detection plots - no flood mask available')

## 8. Comparison with Reference Period

Compare the flood period with the same time in the previous year to assess the anomaly.

In [ ]:
# Compare with reference period if available
if datasets['reference'] is not None and datasets['post_flood'] is not None:
    
    print('Comparing flood period with reference year...')
    
    # Calculate changes relative to reference period
    ref_change_vv = datasets['post_flood'].VV.values - datasets['reference'].VV.values
    ref_change_vh = datasets['post_flood'].VH.values - datasets['reference'].VH.values
    
    # Detect anomalous areas (much different from reference year)
    anomaly_threshold = 4.0  # 4 dB difference from reference
    anomaly_mask_vv = np.abs(ref_change_vv) > anomaly_threshold
    anomaly_mask_vh = np.abs(ref_change_vh) > anomaly_threshold
    anomaly_mask = anomaly_mask_vv | anomaly_mask_vh
    
    # Statistics
    anomaly_pixels = np.sum(anomaly_mask)
    anomaly_percentage = (anomaly_pixels / anomaly_mask.size) * 100
    
    print(f'Anomaly detection (vs. reference year):')
    print(f'  Anomalous pixels: {anomaly_pixels:,}')
    print(f'  Anomaly coverage: {anomaly_percentage:.2f}% of analyzed area')
    
    # Visualize reference comparison
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # Reference year VV
    im1 = axes[0, 0].imshow(datasets['reference'].VV.values, cmap='gray', vmin=-25, vmax=-5)
    axes[0, 0].set_title('Reference VV Backscatter\n(May 2022)')
    axes[0, 0].axis('off')
    cbar1 = plt.colorbar(im1, ax=axes[0, 0], label='Backscatter (dB)')
    
    # 2023 vs 2022 VV change
    im2 = axes[0, 1].imshow(ref_change_vv, cmap='RdBu_r', vmin=-10, vmax=10)
    axes[0, 1].set_title('VV Change vs Reference\n(2023 - 2022)')
    axes[0, 1].axis('off')
    cbar2 = plt.colorbar(im2, ax=axes[0, 1], label='Change (dB)')
    
    # Reference year VH
    im3 = axes[1, 0].imshow(datasets['reference'].VH.values, cmap='gray', vmin=-30, vmax=-10)
    axes[1, 0].set_title('Reference VH Backscatter\n(May 2022)')
    axes[1, 0].axis('off')
    cbar3 = plt.colorbar(im3, ax=axes[1, 0], label='Backscatter (dB)')
    
    # 2023 vs 2022 VH change
    im4 = axes[1, 1].imshow(ref_change_vh, cmap='RdBu_r', vmin=-10, vmax=10)
    axes[1, 1].set_title('VH Change vs Reference\n(2023 - 2022)')
    axes[1, 1].axis('off')
    cbar4 = plt.colorbar(im4, ax=axes[1, 1], label='Change (dB)')
    
    plt.suptitle('Comparison with Reference Year (2022)', fontsize=16, y=0.95)
    plt.tight_layout()
    plt.show()
    
    # Show anomaly detection
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    # Anomaly mask
    im1 = axes[0].imshow(anomaly_mask, cmap='Reds', alpha=0.8)
    axes[0].set_title('Anomalous Areas\n(>4 dB difference from 2022)')
    axes[0].axis('off')
    
    # Combined flood and anomaly
    if flood_mask is not None:
        axes[1].imshow(datasets['post_flood'].VV.values, cmap='gray', vmin=-25, vmax=-5)
        axes[1].imshow(flood_mask, cmap='Blues', alpha=0.5, label='Flood areas')
        axes[1].imshow(anomaly_mask, cmap='Reds', alpha=0.3, label='Anomalous areas')
        axes[1].set_title('Flood + Anomaly Overlay\n(Blue=Flood, Red=Anomaly)')
    else:
        axes[1].imshow(anomaly_mask, cmap='Reds', alpha=0.8)
        axes[1].set_title('Anomalous Areas Only')
    axes[1].axis('off')
    
    plt.tight_layout()
    plt.show()
    
else:
    print('Cannot perform reference comparison - missing reference or post-flood data')

## 9. Summary and Impact Assessment

Provide a comprehensive summary of the flood impact analysis.

In [ ]:
# Generate comprehensive flood impact summary
print('=' * 60)
print('EMILIA ROMAGNA FLOOD IMPACT ASSESSMENT SUMMARY')
print('=' * 60)

print('ANALYSIS DETAILS:')
print(f'  Study area: Emilia Romagna, Italy ({spatial_extent})')
print(f'  Flood event: May 16-17, 2023')
print(f'  Analysis method: Sentinel-1 SAR change detection')
print(f'  Data source: Copernicus Dataspace via openEO')

if flood_mask is not None:
    print('FLOOD DETECTION RESULTS:')
    print(f'  Total analyzed area: ~{lat_range*111:.0f} × {lon_range*85:.0f} km')
    print(f'  Flood-affected pixels: {flood_pixels:,} ({flood_percentage:.2f}%)')
    print(f'  Estimated flood area: {flood_area_km2:.1f} km²')
    
    print('BACKSCATTER CHANGE ANALYSIS:')
    print(f'  VV polarization:')
    print(f'    - Mean change: {np.nanmean(change_vv):.2f} dB')
    print(f'    - Change range: {np.nanmin(change_vv):.2f} to {np.nanmax(change_vv):.2f} dB')
    print(f'    - Pixels below -3 dB threshold: {np.sum(change_vv < -3):,}')
    print(f'  VH polarization:')
    print(f'    - Mean change: {np.nanmean(change_vh):.2f} dB')
    print(f'    - Change range: {np.nanmin(change_vh):.2f} to {np.nanmax(change_vh):.2f} dB')
    print(f'    - Pixels below -2 dB threshold: {np.sum(change_vh < -2):,}')

if 'reference' in datasets and datasets['reference'] is not None and datasets['post_flood'] is not None:
    print('COMPARISON WITH REFERENCE YEAR (2022):')
    print(f'  Anomalous pixels (>4 dB difference): {anomaly_pixels:,} ({anomaly_percentage:.2f}%)')
    print(f'  This indicates areas significantly different from normal conditions')

print('KEY FINDINGS:')
print('1. SAR-based flood detection successfully identified water-covered areas')
print('2. Significant backscatter decreases indicate specular reflection from water')
print('3. VV polarization is more sensitive to open water detection')
print('4. VH polarization helps identify flooded vegetation areas')

print('METHODOLOGY ADVANTAGES:')
print('- Weather-independent monitoring (SAR penetrates clouds)')
print('- Rapid assessment capability for emergency response')
print('- Quantitative change detection with statistical thresholds')
print('- Historical comparison for anomaly detection')

print('APPLICATIONS:')
print('- Emergency response and damage assessment')
print('- Insurance claim validation')
print('- Flood risk mapping and management')
print('- Climate change impact studies')
print('- Infrastructure vulnerability assessment')

print('LIMITATIONS:')
print('- Urban areas may show complex scattering patterns')
print('- Vegetation can mask underlying flood conditions')
print('- Temporal resolution limited by satellite revisit time')
print('- Threshold selection affects detection sensitivity')

print('=' * 60)

In [ ]:
# Clean up downloaded files (optional)
import os

print('\n=== FILE MANAGEMENT ===')
print('Downloaded files:')
total_size = 0
for period_name, filename in downloaded_files.items():
    if filename and os.path.exists(filename):
        file_size = os.path.getsize(filename) / (1024*1024)  # MB
        total_size += file_size
        print(f'  {filename} ({file_size:.1f} MB)')

print(f'Total data downloaded: {total_size:.1f} MB')

# Uncomment the following lines to remove downloaded files
# print('Cleaning up downloaded files...')
# for period_name, filename in downloaded_files.items():
#     if filename and os.path.exists(filename):
#         os.remove(filename)
#         print(f'Removed {filename}')
# print('Cleanup completed')

print('\nFlood impact assessment analysis completed successfully!')
print('\nFor more detailed analysis, consider:')
print('- Integrating with optical imagery for validation')
print('- Using higher resolution SAR data (e.g., TerraSAR-X)')
print('- Incorporating topographic data for flood modeling')
print('- Analyzing multiple flood events for pattern recognition')